In [1]:
# !pip install openai==0.28

In [2]:
##uncomment this to use code in Google Colab
# !pip install fastchat
# !pip install wget
# !pip install evaluate
# !pip install sentence_transformers
# !pip install bert_score
# !pip install datasets
# !pip install sacrebleu

In [3]:
##uncomment this to use code in Google Colab
# from IPython.display import clear_output
# !git clone https://github.com/alfekka/lm-polygraph.git
# %cd lm-polygraph
# %pip install .
# %cd src
# %pip install transformers rouge-score datasets openai
# !curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
# !apt-get install -y nodejs
# %cd lm_polygraph/app
# !npm install
# %cd ../../
# %cd /content/lm-polygraph/src
# %load_ext autoreload
# %autoreload 2

In [4]:
##uncomment this to use code in Google Colab
# %cd ..
# !pwd/
# %cd /content/lm-polygraph/src

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
import os

from lm_polygraph.utils.model import WhiteboxModel
from lm_polygraph.estimators import *
from lm_polygraph.stat_calculators import *
from lm_polygraph.utils.openai_chat import OpenAIChat
from lm_polygraph.utils.deberta import Deberta
from lm_polygraph.utils.deberta import MultilingualDeberta

In [9]:
# model = WhiteboxModel.from_pretrained("IlyaGusev/saiga_llama3_8b")
model = WhiteboxModel.from_pretrained("Vikhrmodels/Vikhr-7B-instruct_0.3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
texts = ["Расскажи биографию Альберта Эйнштейна"]
stat = {}

for calculator in [
    GreedyProbsCalculator(),
    EntropyCalculator(),
    GreedyLMProbsCalculator(),
]:
    stat.update(calculator(stat, texts, model))

claim_extractor=ClaimsExtractor(OpenAIChat("gpt-4o"),language="ru")
stat.update(claim_extractor(stat, texts, model))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
stat.keys()

dict_keys(['input_texts', 'input_tokens', 'greedy_log_probs', 'greedy_tokens', 'greedy_tokens_alternatives', 'greedy_texts', 'greedy_log_likelihoods', 'embeddings_decoder', 'entropy', 'greedy_lm_log_probs', 'greedy_lm_log_likelihoods', 'claims', 'claim_texts_concatenated', 'claim_input_texts_concatenated'])

In [17]:
stat['claims']

[[Claim(claim_text='Альберт Эйнштейн был известным физиком.', sentence='Альберт Эйнштейн был известным физиком, который родился 14 марта 1879 года в Принценовском городе, Саксония, Германия', aligned_token_ids=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]),
  Claim(claim_text='Альберт Эйнштейн родился 14 марта 1879 года.', sentence='Альберт Эйнштейн был известным физиком, который родился 14 марта 1879 года в Принценовском городе, Саксония, Германия', aligned_token_ids=[7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 25, 26, 27, 29, 30, 31, 32, 33]),
  Claim(claim_text='Альберт Эйнштейн родился в Принценовском городе.', sentence='Альберт Эйнштейн был известным физиком, который родился 14 марта 1879 года в Принценовском городе, Саксония, Германия', aligned_token_ids=[7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 34, 35, 36, 37, 38, 39, 40]),
  Claim(claim_text='Альберт Эйнштейн родился в Саксонии.', sentence='Альберт Эйнштейн был известным физиком, который родился 14 марта 1879 года в Принценовском 

In [18]:
print("Output:", stat["greedy_texts"][0])

Output: .<|im_start|> 
<|im_end|> assistant
Альберт Эйнштейн был известным физиком, который родился 14 марта 1879 года в Принценовском городе, Саксония, Германия. Его родители были Исаак Эйнштейн и Паула Канович. Он был старшим из двух детей в семье.

Эйнштейн был очень умным ребенком, и его родители знали, что


In [23]:
print("Output:", stat["greedy_texts"][0])
print()
for claim in stat["claims"][0]:
    print("claim:", claim.claim_text)
    print("aligned tokens:", claim.aligned_token_ids)
    print()

Output: .<|im_start|> 
<|im_end|> assistant
Альберт Эйнштейн был известным физиком, который родился 14 марта 1879 года в Принценовском городе, Саксония, Германия. Его родители были Исаак Эйнштейн и Паула Канович. Он был старшим из двух детей в семье.

Эйнштейн был очень умным ребенком, и его родители знали, что

claim: Альберт Эйнштейн был известным физиком.
aligned tokens: [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

claim: Альберт Эйнштейн родился 14 марта 1879 года.
aligned tokens: [7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 25, 26, 27, 29, 30, 31, 32, 33]

claim: Альберт Эйнштейн родился в Принценовском городе.
aligned tokens: [7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 34, 35, 36, 37, 38, 39, 40]

claim: Альберт Эйнштейн родился в Саксонии.
aligned tokens: [7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 42, 43, 44, 45]

claim: Альберт Эйнштейн родился в Германии.
aligned tokens: [7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 47, 48]

claim: Его родители были Исаак Эйнштейн.
aligned tokens: [50, 51, 52

In [24]:
# Maximum Claim Probability
max_prob = MaximumClaimProbability()
max_prob(stat)  # Uncertainty for each claim, the higher, the less certain

[[4.2850304,
  4.2913685,
  9.6921425,
  4.8257813,
  4.0302954,
  6.2322206,
  5.2375154,
  4.74283,
  2.4657412]]

In [28]:
# Perlexity
perlexity_claim = PerplexityClaim()
perlexity_claim (stat)

[[0.3060736,
  0.23840936,
  0.57012606,
  0.34469867,
  0.33585796,
  0.5665655,
  0.7482165,
  0.7904716,
  0.49314824]]

In [29]:
# Maximum Token Entropy
max_token_ent = MaxTokenEntropyClaim()
max_token_ent(stat)

[[5.4448175,
  6.3854904,
  6.3854904,
  6.3854904,
  6.3854904,
  5.713653,
  5.364495,
  5.364495,
  5.364495]]

In [30]:
# Pointwise Mutual Information
pmi = PointwiseMutualInformationClaim()
pmi (stat)

[[-8.520321387375589,
  -10.170895054841822,
  -12.592961494272458,
  -10.221559652520227,
  -10.866402111874777,
  -4.133144408937369,
  -90.55689358944073,
  -79.78021109336987,
  -65.40558728808537]]

In [31]:
p_true_calculator_stat = PromptCalculator(
    "Question: {q}\n Possible answer:{a}\n "
    "Is the possible answer True or False? The possible answer is: ",
    "True",
    "p_true_claim",
    input_text_dependency="claim_input_texts_concatenated",
    sample_text_dependency=None,
    generation_text_dependency="claim_texts_concatenated",
)

stat.update(p_true_calculator_stat(stat, texts, model))

ptrue_claim = PTrueClaim()
ptrue_claim (stat)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[5.4730864,
  5.5996523,
  5.2887564,
  5.383123,
  4.894233,
  7.15702,
  6.41344,
  6.4006567,
  7.5723834]]

In [36]:
# Claim Conditional Probability
for calculator in [
    GreedyAlternativesNLICalculator(MultilingualDeberta())
]:
    stat.update(calculator(stat, texts, model))

ccp = ClaimConditionedProbabilityClaim()
ccp (stat)

tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

[[-0.4713349140969484,
  -0.4177353638074024,
  -0.16863406344265375,
  -0.44496233440480837,
  -0.667818389405516,
  -0.9047091111261258,
  -0.75010673778448,
  -0.9998908271780086,
  -0.7692339308543321]]

In [37]:
for calculator in [
    GreedyAlternativesFactPrefNLICalculator(MultilingualDeberta())
]:
    stat.update(calculator(stat, texts, model))

ccp_no_cxt=ClaimConditionedProbabilityClaim(nli_context="fact_pref")
ccp_no_cxt (stat)

[[-0.8267625446961321,
  -0.3563652294162695,
  -0.532586907787179,
  -0.4886474033583824,
  -0.9094577054754234,
  -0.21712703930398927,
  -0.7713261913758136,
  -0.8904668712710353,
  -0.8662042533652458]]

In [38]:
from lm_polygraph.generation_metrics.openai_fact_check import OpenAIFactCheck
checker = OpenAIFactCheck('gpt-4o', language="ru")
chatgpt_response = checker(stat, None, None)

In [39]:
chatgpt_response

[[0, 0, 1, 1, 0, 1, 0, 0, 1]]